# Python and Data Structures with Python

# **Assignment 4**

#### **Submitted by:**  Arushi Marwaha

#### **Course:** MSc Data Science 1  

#### **Roll:** MDS202512

## Code for creating the Bus Booking system:

In [28]:
class BusBooking:
   
    """A class to simulate a simple bus booking system with seat preferences and waiting list management."""

    # --------------------------------------------Constructor 

    def __init__(self):
        """Constructor for initializing the bus and booking records."""
        
        # initializing the bus seat layout
        self.bus = {}  # creating dictionary to store seat assignments
        
        # assigning seat numbers (W1–W20 for Window, A1–A20 for Aisle)
        for i in range(1, 21):
            self.bus['W' + str(i)] = None
            self.bus['A' + str(i)] = None

        # initializing control variables and data structures
        self.next_booking_id = 1  # generating unique booking IDs
        self.booking_info = {}    # mapping booking IDs to passenger details
        self.waiting_list = []    # queue to handle overflow bookings

    
    # --------------------------------------------Book function 

    def book(self, name, preference=None):
        """Function for booking a seat based on user preference."""
        # generating a unique booking ID
        booking_id = self.next_booking_id
        self.next_booking_id += 1

        # initializing seat assignment before processing
        seat_assigned = None

        # handling missing preference and making it case-insensitive
        if preference is None:
            preference = ''
        else:
            preference = preference.lower()

        # assigning a window seat if available and preferred
        if preference in ['w', 'window']:
            for i in range(1, 21):
                if self.bus['W' + str(i)] is None:
                    seat_assigned = 'W' + str(i)
                    self.bus[seat_assigned] = name
                    break

        # assigning an aisle seat if available and preferred
        elif preference in ['a', 'aisle']:
            for i in range(1, 21):
                if self.bus['A' + str(i)] is None:
                    seat_assigned = 'A' + str(i)
                    self.bus[seat_assigned] = name
                    break

        # assigning any available seat if preference cannot be met
        if seat_assigned is None:
            for i in range(1, 21):
                if self.bus['W' + str(i)] is None or self.bus['A' + str(i)] is None:
                    seat_assigned = 'W' + str(i) if self.bus['W' + str(i)] is None else 'A' + str(i)
                    self.bus[seat_assigned] = name
                    break

        # adding to waiting list if all seats are occupied
        if seat_assigned is None:
            self.waiting_list.append((booking_id, name))  # storing booking in waiting queue
            wait_pos = len(self.waiting_list)             # determining position in waiting list
            outcome = 'WL-' + str(wait_pos)               # setting waiting list status
            self.booking_info[booking_id] = {'name': name, 'seat': outcome}
            return (booking_id, outcome)

        # storing confirmed booking information
        self.booking_info[booking_id] = {'name': name, 'seat': seat_assigned}
        return (booking_id, seat_assigned)

    
    
    # --------------------------------------------Cancel function 

    def cancel(self, booking_id):
        """Function for cancelling an existing booking."""
        # checking if the provided booking ID exists
        if booking_id not in self.booking_info:
            return False

        # retrieving and removing the booking record
        booking = self.booking_info.pop(booking_id)
        seat = booking['seat']

        # handling cancellation from waiting list
        if seat.startswith('WL'):
            # removing from waiting list
            self.waiting_list = [w for w in self.waiting_list if w[0] != booking_id]

            # updating waiting list numbering
            for i, (b, name) in enumerate(self.waiting_list):
                self.booking_info[b]['seat'] = 'WL-' + str(i + 1)
            return True

        # freeing the booked seat for others
        self.bus[seat] = None

        # if waiting list has pending passengers, reassigning freed seat
        if len(self.waiting_list) > 0:
            next_b, next_name = self.waiting_list.pop(0)
            self.bus[seat] = next_name
            self.booking_info[next_b]['seat'] = seat

            # renumbering waiting list after promotion
            for i, (b, name) in enumerate(self.waiting_list):
                self.booking_info[b]['seat'] = 'WL-' + str(i + 1)
        return True

    
    # --------------------------------------------Status function 

    def status(self, booking_id):
        """Function for checking the booking status of a passenger."""
        # verifying if the booking ID exists
        if booking_id in self.booking_info:
            # retrieving passenger info and seat assignment
            info = self.booking_info[booking_id]
            return (info['name'], info['seat'])
        # returning None for invalid booking IDs
        return None
    
    
    # --------------------------------------------String Represenation 

    def __str__(self):
        """Function for displaying all bookings in a readable format."""
        
        # sorting booking IDs for cleaner display
        sorted_booking_ids = sorted(self.booking_info.keys())
        result_list = []

        # preparing a structured list of all bookings
        for booking_id in sorted_booking_ids:
            info = self.booking_info[booking_id]
            result_list.append((booking_id, info['name'], info['seat']))

        # returning string representation of booking summary
        return str(result_list)


## Implementation:


In [32]:

b = BusBooking()

print("\n Initial Bookings ")
print(b.book("Arushi", "window"))     #  W1
print(b.book("Sanwaya", "aisle"))     #  A1
print(b.book("Unnati", "window"))     #  W2
print(b.book("Aarushi", "aisle"))     #  A2
print(b.book("Shreya"))               #  next available (W3)
print(b.book("Inni", "w"))            #  next window (W4)
print(b.book("Sania", "a"))           #  next aisle (A3)
print(b.book("Nandini", "window"))    #  W5
print(b.book("Deepta", "aisle"))      #  A4
print(b.book("Aryan"))                #  next available seat
print(b.book("Radhika"))              #  next available seat

print("\nCurrent Bookings (partial bus fill):")
print(b,"\n")

# Filling rest of the bus (up to 40 seats)
for i in range(12, 41):
    b.book(f"Passenger {i}")

print(f"\nTotal Active Bookings (including all seats): {len(b.booking_info)}")

# Adding extra passengers to start waiting list
print(b.book("Waiter1"))
print(b.book("Waiter2"))
print(b.book("Waiter3"))

print("\nWaiting List Entries:")
for bid, info in b.booking_info.items():
    if info['seat'].startswith('WL'):
        print(bid, info)

# Cancel a booked seat to trigger seat reassignment
print("\n  Cancellation of a Booked Seat ")
print("Cancelling Arushi's booking (ID = 1, W1):", b.cancel(1))

print("\nAfter Cancellation:")
print("First waiting passenger (Waiter1) should now occupy W1.")
for bid, info in b.booking_info.items():
    if info['name'] == "Waiter1":
        print( bid, info)

# Cancel a waiting list passenger
print("\n  Cancelling a Waiting List Passenger  ")
for bid, info in b.booking_info.items():
    if info['name'] == "Waiter3":
        wl3_id = bid
        break
print("Cancelling Waiter3:", b.cancel(wl3_id))

# Trying to cancel a non-existent booking ID
print("\nAttempting to cancel a non-existent booking ID (e.g., 2399):")
print("Result:", b.cancel(2399))   # should return False


print("\nRemaining Waiting List (renumbered):")
for bid, info in b.booking_info.items():
    if info['seat'].startswith('WL'):
        print(bid, info)

# checking the status of a few passengers
print("\n Checking Status of Some Bookings ")
print("Sanwaya:", b.status(2))
print("Waiter1:", b.status(41))
print("Waiter2:", b.status(42))
print("Waiter3:", b.status(43))

# Final state
print("\n Final bookings done")
print(b)



 Initial Bookings 
(1, 'W1')
(2, 'A1')
(3, 'W2')
(4, 'A2')
(5, 'W3')
(6, 'W4')
(7, 'A3')
(8, 'W5')
(9, 'A4')
(10, 'A5')
(11, 'W6')

Current Bookings (partial bus fill):
[(1, 'Arushi', 'W1'), (2, 'Sanwaya', 'A1'), (3, 'Unnati', 'W2'), (4, 'Aarushi', 'A2'), (5, 'Shreya', 'W3'), (6, 'Inni', 'W4'), (7, 'Sania', 'A3'), (8, 'Nandini', 'W5'), (9, 'Deepta', 'A4'), (10, 'Aryan', 'A5'), (11, 'Radhika', 'W6')] 


Total Active Bookings (including all seats): 40
(41, 'WL-1')
(42, 'WL-2')
(43, 'WL-3')

Waiting List Entries:
41 {'name': 'Waiter1', 'seat': 'WL-1'}
42 {'name': 'Waiter2', 'seat': 'WL-2'}
43 {'name': 'Waiter3', 'seat': 'WL-3'}

  Cancellation of a Booked Seat 
Cancelling Arushi's booking (ID = 1, W1): True

After Cancellation:
First waiting passenger (Waiter1) should now occupy W1.
41 {'name': 'Waiter1', 'seat': 'W1'}

  Cancelling a Waiting List Passenger  
Cancelling Waiter3: True

Attempting to cancel a non-existent booking ID (e.g., 2399):
Result: False

Remaining Waiting List (renu

----------------------------------------------------------------------------------------------------------------------